In [74]:
import pandas as pd
import numpy as np
import warnings

from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score
from sklearn.feature_selection import RFE

from sys import platform

if platform == "win32":
    path = 'C:/Users/olive/GitHub/f1-analytics/'
elif platform == "darwin":
    path = '~/Documents/GitHub/f1-analytics/'
    # path = '/Users/oliverjcarter/Documents/GitHub/f1-analytics/'

warnings.filterwarnings("ignore", category=RuntimeWarning) 
pd.options.mode.chained_assignment = None  # default='warn'

%matplotlib inline

In [75]:
data = pd.read_csv(path+'data/ml_input.csv')
parameters = pd.read_csv(path+'parameters/svm_classifier.csv')

In [76]:
### Season to test results

N = 2021

In [77]:
params_to_drop = ['season', 'round', 'driver', 'constructor', 'circuit_id', 'podium', 'driver_points_from']

In [78]:
df = data.copy()

train = df[df.season < N]
train.podium = df.podium.map(lambda x: 1 if x == 1 else 0)

scaler = StandardScaler()
X_train = train.drop(params_to_drop, axis=1)
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)

y_train = np.asarray(train.podium.values)

In [79]:
def get_predictions(model, X_train, y_train):
    predictions = []
    prob = []
    
    for circuit in df[df.season == N]['round'].unique():

        test = df[(df.season == N) & (df['round'] == circuit)]
        X_test = test.drop(params_to_drop, axis=1)
        y_test = test.podium

        # Scaling
        X_test = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

        # Make Predictions
        prediction_df = pd.DataFrame(model.predict_proba(X_test), columns=['proba_0', 'proba_1'])
        prediction_df['actual'] = y_test.reset_index(drop=True)
        prediction_df['predicted'] = prediction_df.proba_1.map(lambda x: 1 if x == prediction_df.proba_1.max() else 0)

        predictions += list(prediction_df['predicted'].values)  
        prob += list(prediction_df['proba_1'].values)

        # Retrain model
        # X_train = pd.concat([X_train, X_test])
        # y_train = np.append(y_train, y_test)

        # model.fit(X_train, y_train)

    return predictions, prob

In [80]:
parameters.head()

,model,gamma,C,kernel,score
0,svm_classifier,0.100000,10.000000,rbf,0.736842
1,svm_classifier,0.100000,6.951928,rbf,0.736842
2,svm_classifier,0.069519,10.000000,rbf,0.736842
3,svm_classifier,0.000428,2.335721,linear,0.684211
4,svm_classifier,0.000298,2.335721,linear,0.684211


In [81]:
params = parameters.iloc[0]

gamma = params.gamma
c = params.C
kernel = params.kernel

params

model     svm_classifier
gamma                0.1
C                   10.0
kernel               rbf
score           0.736842
Name: 0, dtype: object

In [82]:
test = df[(df.season == N)]

model = SVC(probability=True, gamma=gamma, C=c, kernel=kernel)
model.fit(X_train, y_train)

predictions, probs = get_predictions(model, X_train, y_train)


test['predicted_winner'] = predictions
test['probability'] = probs

In [83]:
rounds = test['round'].nunique()

print(f'Outright Winner Accuracy: { round(test.query("predicted_winner == 1 & podium == 1").shape[0] / rounds, 2) }')

print(f'Top-Two Accuracy: { round(test.query("predicted_winner == 1 & podium <= 2").shape[0] / rounds, 2) }')

print(f'Podium Accuracy: { round(test.query("predicted_winner == 1 & podium <= 3").shape[0] / rounds, 2) }')

Outright Winner Accuracy: 0.74
Top-Two Accuracy: 0.84
Podium Accuracy: 0.95


In [87]:
test.query("predicted_winner == 1 & podium == 1")
rounds

19

In [84]:
def highlight_row(row):
    if row['podium'] == 1:
        return ['background-color: limegreen'] * len(row)
    elif 1 < row['podium'] <= 3:
        return ['background-color: yellow; opacity: .95; color: black'] * len(row)
    elif 3 < row['podium'] <= 10:
        return ['background-color: orange'] * len(row)
    elif row['podium'] > 10:
        return ['background-color: #E34234'] * len(row)

In [85]:
q = test.query('predicted_winner == 1')[['circuit_id', 'driver', 'podium', 'starting_grid', 'probability']]

q.style.set_properties(color='white').apply(highlight_row, axis=1)

,circuit_id,driver,podium,starting_grid,probability
2764,bahrain,lewis_hamilton,1,2,0.044949
2785,imola,lewis_hamilton,2,1,0.076145
2806,portimao,valtteri_bottas,3,1,0.044725
2824,catalunya,lewis_hamilton,1,1,0.332615
2844,monaco,max_verstappen,1,2,0.060279
2865,baku,pierre_gasly,3,4,0.045412
2883,ricard,max_verstappen,1,1,0.371076
2902,red_bull_ring,max_verstappen,1,1,0.232066
2922,red_bull_ring,max_verstappen,1,1,0.120340
2950,hungaroring,max_verstappen,9,3,0.052198
